In [74]:
import re
import json
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import collections

In [159]:
import pymongo
import json
from Processing.Processing import Processing

client = pymongo.MongoClient()
db = client["news"]
print(client['news'].list_collection_names())
coll_raw = db['thestar_v1_covid']
coll_processed = db['sebenarnya_v2_proccessed1']

['thestar_v1', 'thestar_v1_covid', 'sebenarnya_v2_proccessed2', 'sebenarnya_v2_proccessed1', 'sebenarnya_v2_test1', 'sebenarnya_v1_test2']


In [170]:
# coll_raw.delete_many({})
len(list(coll_raw.find()))

300

In [171]:
list(coll_raw.find())[299]

{'_id': ObjectId('5e8b5d1f81ef25576e5b1672'),
 'full_html': '<div class="articleDetails focus-wrapper " href="https://www.thestar.com.my/news/nation/2020/04/06/83-million-households-singles-to-receive-bpn-aid-from-monday-april-6" data-widget-id="1341" data-articleid="3115188" data-articletitle="8.3 million households, singles to receive BPN aid from Monday (April 6)">\n    <div id="pb" data-cms_article_id="3115188" data-np_article_id="253148"></div>\n    \n        <div class="headline story-pg">\n        <h1>\n            8.3 million households, singles to receive BPN aid from Monday (April 6)\n        </h1>\n        <hr>\n    </div>\n    \n    <style type="text/css">\n        @media  only screen and (max-width: 960px){\n        #sideNote {\n        position: initial !important;\n        }\n        }\n        .story-wrapper {\n            padding-top: 0 !important;\n        }\n\n        .story-set-gallery img {\n            position: absolute;\n            height: 100%;\n            wi

In [58]:
#with audio
https://sebenarnya.my/dakwaan-nota-suara-anggota-atm-pukul-individu-yang-keluar-rumah-ketika-pkp-adalah-palsu/
    
#with caption
https://sebenarnya.my/kilang-assb-di-taman-perindustrian-malaysia-china-kuantan-mckip-terbakar/

786

In [73]:
import pymongo
import json
from Processing.Processing import Processing

client = pymongo.MongoClient()
db = client["news"]
print(client['news'].list_collection_names())
coll_raw = db['sebenarnya_v2_test1']
coll_processed = db['sebenarnya_v2_proccessed1']

['sebenarnya_v2_proccessed2', 'sebenarnya_v2_proccessed1', 'sebenarnya_v2_test1', 'sebenarnya_v1_test2']


In [ ]:
# coll_processed.delete_many({})
list(coll_processed.find())[111]


In [8]:
for raw_dict in list(coll_raw.find())[1:]:
    cls = Processing(raw_dict)
    coll_processed.insert_one(cls.json_file)

In [3]:
x = coll_raw.find_one({'title':'4 Nota Suara Yang Menggunakan Nama MKN Dakwa Darurat Akan Diisytihar Adalah Palsu'})
x['url']

'https://sebenarnya.my/4-nota-suara-yang-menggunakan-nama-mkn-dakwa-darurat-akan-diisytihar-adalah-palsu/'

In [7]:
db_dict = raw_dict

title = db_dict['title']
content = db_dict['content_html']
soup = BeautifulSoup(content[0], 'html.parser')
rm_index = soup.text.find(soup.find('div',{'class':'awac-wrapper'}).text) if soup.find('div',{'class':'awac-wrapper'}) else len(soup.text)
all_text = soup.text[:rm_index]
lines = [line.strip() for line in all_text.split('\n') if line.strip() != '']

In [10]:
cls = Processing(db_dict)

['SEBENARNYA:', 'SUMBER:'] 4 Nota Suara Yang Menggunakan Nama MKN Dakwa Darurat Akan Diisytihar Adalah Palsu


In [12]:
cls.title

'4 Nota Suara Yang Menggunakan Nama MKN Dakwa Darurat Akan Diisytihar Adalah Palsu'

In [10]:
' '.join(all_text.split('\n')).strip()

'SEBENARNYA: Jabatan Kesihatan Negeri Terengganu ingin merujuk kepada satu kenyataan yang tular di aplikasi WhatsApp berkenaan kononnya terdapat 30 kes COVID-19 di Durian Burung, Kuala Terengganu berpunca daripada majlis tahlil kematian dan 30 orang ahli keluarga telah dikuarantin di Chendering. Pihak jabatan ini menafikan sepenuhnya kenyataan tersebut. Sehingga 3 April 2020, tiada kes COVID-19 yang dikesan di Kg Durian Burung, Kuala Terengganu. SUMBER: Facebook Rasmi JKNT'

In [43]:
import requests
from bs4 import BeautifulSoup

In [44]:

url = "https://www.thestar.com.my/news/true-or-not/"
getreq = requests.get(url)
firstsoup = BeautifulSoup(getreq.text,"html.parser")
# ## find the viewstate & event validation that is required for extracting POST REQUEST FOR page >2
# VIEWSTATE = firstsoup.find('input',{'id':'__VIEWSTATE'}).get('value')
# EVENTVALIDATION = firstsoup.find('input',{'id':'__EVENTVALIDATION'}).get('value')
# ## function to get the details of the other pages (page>2)
# def get_each_page(input_page):
#     data = {
#         "__EVENTTARGET": "ctl00$MainContent$gvReport",
#         "__EVENTARGUMENT": input_page,
#         "__EVENTVALIDATION":EVENTVALIDATION,
#         "__VIEWSTATE":VIEWSTATE,
#            }
#     r = requests.post(url,data=data)
#     soup=BeautifulSoup(r.text,"html.parser")
#     return soup

In [71]:
firstsoup.find('div', {'class': 'button-view btnLoadMore'})

<div class="button-view btnLoadMore" style="margin: 10px auto 15px;">
<a id="loadMorestories">Load more </a>
</div>

In [ ]:
data = {
    "__EVENTTARGET": "ctl00$MainContent$gvReport",
    "__EVENTARGUMENT": input_page,
    "__EVENTVALIDATION":EVENTVALIDATION,
    "__VIEWSTATE":VIEWSTATE,
       }

In [72]:
!pip install scrapy-splash

In [64]:
firstsoup.find('input',{'id':'__VIEWSTATE'}).get('value')

AttributeError: 'NoneType' object has no attribute 'get'